In [ ]:
[toc]

# TensorFlow 中的几个关键概念：Tensor，Operation，Graph，Session

前言：TensorFlow是一种符号式编程框架，首先要构造一个图（graph），然后在这个图上做运算。打个比方，graph就像一条生产线，session就像生产者。生产线具有一系列的加工步骤（加减乘除等运算），生产者把原料投进去，就能得到产品。不同生产者都可以使用这条生产线，只要他们的加工步骤是一样的就行。同样的，一个graph可以供多个session使用，而一个session不一定需要使用graph的全部，可以只使用其中的一部分。

## 一、Tensorflow的工作流程

1.  根据需求，创建计算图Graph
2.  开启会话Session，读取数据运行Graph
3.  获取结果

![](https://pic4.zhimg.com/v2-e922ec623e1c8e0d3721d6f8e37daa53_b.jpg)


## **二、TensorFlow 几个关键概念：Tensor，Operation，Graph，Session**

### 1. Tensor（数据节点）

（1）Tensor定义

在 TensorFlow 中，所有在节点之间传递的数据都为 Tensor 对象。

Tensor定义：A Tensor is a symbolic handle to one of the outputs of an Operation. **It does not hold the values of that operation’s output**, but instead provides a means of computing those values in a TensorFlow tf.Session

也就是说，Tensor本身是不存储数据的，创建一个Tensor实际就是声明了一个数据节点。只有开启Session进行运算的时候，才会获取到数据。

（2）Tensor的阶

如下图所示，Rank为0、1、2时分别称为标量、向量和矩阵，Rank为3时是3阶张量，Rank大于3时是N阶张量。这些标量、向量、矩阵和张量里的元素可以是数组，可以是tensor或者其他python基本数据类型。当所有元素类型一致且均为tensor（数组）时，则可将Rank的阶数当做tensor（数组）的阶数。通常我们讨论的数据类型是指元素的数据类型。

![](https://pic4.zhimg.com/v2-d4877269be47ca75b734931aee0ddadb_b.jpg)
Tensor的阶数

（3）Tensor的几个重要属性

shape：类似于Numpy中ndarray.shape，比方说一个2行3列的二维矩阵，他的形状就是2行3列。

dtype：类似于Numpy中ndarray.dtype，常用的类型有：

> tf.uint8: 8\-bit unsigned integer.
> tf.int32: 32\-bit signed integer.
> tf.int64: 64\-bit signed integer.
> tf.String: String.
> tf.float32: 32\-bit single\-precision floating\-point.
> tf.float64: 64\-bit double\-precision floating\-point.

name：每个Tensor都必须有name属性。在创建Tensor时，如果用户没有指定name，Tensorflow会自动设置；在同一张Graph中，不会有Tensor重名，当用户设定的name重名时，Tensorlfow会自动加入后缀进行区分。

（4）几种Tensor

*   **常量Tensor**：值不能改变,最常见的常量创建方式为tf.constant(value, dtype=None, shape=None, name="Const", verify\_shape=False),其中value不可少，verify\_shape表示常量的形状是否可以被更改，默认不可更改。初此之外还有以下常量的创建方法：

```
# 产生全 0 的张量
tensor_a = tf.zeros(shape=[3,4], dtype=tf.float32, name=None)
a = tf.zeros_like(tensor_a, dtype=None, name=None)#tensor_a with all elements set to zero.
# 产生全 1 的张量
tensor_b = tf.ones(shape=[3,4], dtype=tf.float32, name=None)
b = tf.ones_like(tensor_b, dtype=None, name=None)
# Creates a tensor of shape  and fills it with value
#tf.fill(dims, value, name=None)
tf.fill([2, 3], 9) ==> [[9, 9, 9]
                        [9, 9, 9]]
# 产生常量 Tensor, value 值可为 python 标准数据类型、Numpy 等
tf.constant(-1.0, shape=[2, 3]) => [[-1., -1., -1.]  # Note: 注意 shape 的用法(广播机制)
                                    [-1., -1., -1.]]
tf.constant([1,2,3,4,5,6], shape=[2,3]) => [[1, 2, 3]
                                            [4, 5, 6]]
```

In [1]:
注意：常量tensor无需初始化，可以直接获取tensor的值。
但仍然遵循tensorflow的运行机制，即数据只会在session中流动，
即无论是初始化赋值还是取出tensor的值都必须在session中执行


#### 变量Tensor

值可以改变，**可训练**。在神经网络中，变量一般可作为储存权重和其他信息的矩阵，而常量可作为储存超参数或其他结构信息的变量。

SyntaxError: invalid character in identifier (<ipython-input-1-b886adc6f38d>, line 1)

#### 变量初始化

必须初始化变量（将初始值initial_value传给tensor）后才能取出tensor的值，常见的初始化方法如下：

1. 全局变量初始化(其实全局变量初始化方法就是将初始值通过变量的assign方法向变量分配值)
2. 将一个变量的值赋值给另一个变量；
3. 运行变量的初始化函数variable.initializer();
4. restore恢复变量值

In [2]:
#1.全局变量初始化：tf.global_variables_initializer()
import tensorflow as tf

weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35), name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")

# Add an Op to initialize global variables.
init_op = tf.global_variables_initializer()

# Launch the graph in a session.
with tf.Session() as sess:
    # Run the Op that initializes global variables.
    sess.run(init_op)
    print(weights.eval(), biases.eval())#初始化成功才能取出tensor的值

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/frame

[[-0.30484375 -0.28239942 -0.24617383 ...  0.03378935  0.23692106
  -0.30959684]
 [ 0.02124093 -0.56106323 -0.4319002  ... -0.17990465  0.25717613
  -0.75606394]
 [ 0.06222272  0.46132556  0.16025361 ... -0.3126479   0.01613351
   0.3921509 ]
 ...
 [-0.39207277 -0.21199372  0.44391528 ...  0.32433954 -0.22668724
   0.37477973]
 [ 0.29704857 -0.5534732  -0.17758574 ...  0.16526787 -0.50276715
  -0.21533553]
 [-0.68663037  0.02550755  0.6820415  ...  0.14728025 -0.24712875
   0.6034345 ]] [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/te

In [3]:
#2.将一个变量的值赋值给另一个变量：initialized_value()
import tensorflow as tf

w = tf.Variable(tf.ones(shape=[3]), name='w')
w2 = tf.Variable(w.initialized_value(), name='w2')#用已经初始化的w的值作为w2的初始值
w_twice = tf.Variable(w.initialized_value() * 2, name='w_twice')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())#注意这里必须使用全局变量初始化。w，w2，w_twice都只是计算图中的一个op
    print(w.eval(), w2.eval(), w_twice.eval())#初始化成功才能取出tensor的值

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
[1. 1. 1.] [1. 1. 1.] [2. 2. 2.]


In [4]:
#3.运行变量的初始化函数：variable.initializer()
import tensorflow as tf

w = tf.Variable(tf.ones(shape=[3]), name='w')
with tf.Session() as sess:
    sess.run(w.initializer)#仅仅初始化w本身，等价于w.initializer.run()
    print(w.eval())

[1. 1. 1.]


In [5]:
#4.restore恢复变量值
import tensorflow as tf

#首先恢复graph
saver = tf.train.import_meta_graph('./checkpoints/model.ckpt-999.meta')
with tf.Session() as sess:
    #恢复最新保存的权重
    saver.restore(sess, tf.train.latest_checkpoint('./checkpoints'))
    #指定一个权重恢复
    saver.restore(sess, './checkpoints/model.ckpt-999')#注意不要加文件后缀名

OSError: File ./checkpoints/model.ckpt-999.meta does not exist.


### 2. Operation（计算节点）

将多个Tensor连接在一起，形成新的Tensor。例如tf.add、tf.mul等操作。tensor在graph上也是一个Operation（简称op）。但凡是op，都需要通过session运行之后，才能得到结果。

### 3.Graph（数据节点+计算节点）

![](https://pic2.zhimg.com/v2-73c7c6bf327df8edd30cb8dbbd084589_b.jpg)

Graph就是由一系列op构成的。具体来说就是由W、b、x等数据节点及Add、MatMul等计算节点共同构成一个Graph。

在Tensorflow中，始终存在一个默认的Graph。如果要将Operation添加到默认Graph中，只需要调用定义Operation的函数（例如tf.add()）。

如果我们需要定义多个Graph，则需要在with语句中调用Graph.as_default()方法将某个graph设置成默认Graph，于是with语句块中调用的Operation或Tensor将会添加到该Graph中。

In [ ]:
import tensorflow as tf

#定义一个图：只有一个图时，这个图就是默认图，所有操作都自动添加到这个图中
g = tf.Graph()#tensorflow会默认给我们建立一个graph,这句话可以省略
a = tf.constant(2)#将Operation添加到默认Graph中，只需要调用定义Operation的函数

print(a.graph)
print(tf.get_default_graph()) # 通过调用tf.get_default_graph()访问默认创建的图的位置

#定义多个图：需要声明某个操作是定义在哪个图中的
g1 = tf.Graph()
g2 = tf.Graph()

#将某个graph设置成默认Graph，with语句块中调用的Operation或Tensor将会添加到该Graph中
with g1.as_default():
    x = tf.constant(2)
    y = tf.constant(3)
    z = tf.add(x, y)
    print(x.graph, y.graph, z.graph)
    print(tf.get_default_graph())

with g2.as_default():
    v = tf.constant(4)
    u = tf.add(2, v)
    print(v.graph, u.graph)
    print(tf.get_default_graph())

#e不是定义在with语句里面的,e会包含在tensorflow默认创建的图中。也就是说e与a在同一个图中
e=tf.constant(value=15)
print(e.graph)

如果在创建Session时没有指定Graph，则该Session会加载默认Graph。如果创建了多个Graph，则需要创建不同的Session来加载每个Graph，而每个Graph则可以加载在多个Session中进行计算。

In [ ]:
import tensorflow as tf
g1 = tf.Graph()
with g1.as_default():
    c1 = tf.constant([1.0])
    

with tf.Graph().as_default() as g2:
    c2 = tf.constant([2.0])

with tf.Session(graph=g1) as sess1:
    print(sess1.run(c1))
    
with tf.Session(graph=g2) as sess2:
    print(sess2.run(c2))

# result:
# [ 1.0 ]
# [ 2.0 ]
#如果将上面例子的sess1.run(c1)和sess2.run(c2)中的c1和c2交换一下位置，运行会报错。
# 因为sess1加载的g1中没有c2这个Tensor，同样地，sess2加载的g2中也没有c1这个Tensor。

### 4.Session（对Graph进行计算）

Tensorflow先构造Graph，然后开启session在这个Graph上做运算。Graph是由一系列op组成。但凡是op，都需要通过session运行之后，才能得到结果。Session的作用就是执行Operation（Tensor也可以看作一种Operation）。

**执行Operation有两种方式：**

*   调用Session.run()方法： 该方法的定义如下所示，参数fetches便是一个或者多个Operation或者Tensor。

```text
tf.Session.run(fetches, feed_dict=None)
```

*   调用Tensor.eval()方法： 这个方法接收参数session，用于指定在哪个session中计算。该参数默认为None，表示在默认session中计算。设置默认session有两种方式：

In [ ]:
#设置默认session的方式一
import tensorflow as tf

a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
c = tf.add(a, b)

with tf.Session() as sess:
    print(c.eval())

In [6]:
#设置默认session的方式二
import tensorflow as tf

a = tf.constant([1.0, 2.0])
b = tf.constant([3.0, 4.0])
c = tf.add(a, b)

sess = tf.Session()
with sess.as_default():
    print(c.eval())
sess.close()

[4. 6.]


**session.run()与tensor.eval()都是开启对Graph的计算，下面比较一下两者：**

首先，tensor.eval()只适用于tensor。而session.run()不仅适用于tensor，还可用于没有输出的op。对于tensor，调用session.run()与tensor.eval()是等价的。

In [ ]:
import tensorflow as tf
t = tf.constant(42.0)
sess = tf.Session()
# calling t.eval() is equivalent to calling tf.get_default_session().run(t).
with sess.as_default():   # or `with sess:` to close on exit
    assert sess is tf.get_default_session()
    assert t.eval() == sess.run(t)
sess.close()

其次，你可以使用session.run()在同一步骤中获取许多张量的值，而tensor.eval()却只能一次获得一个张量的值。

In [7]:
import tensorflow as tf

t = tf.constant(42.0)
u = tf.constant(37.0)
tu = tf.add(t, u)
ut = tf.add(u, t)

sess = tf.Session()
with sess.as_default():
   tu.eval()  # runs one step
   ut.eval()  # runs one step
   sess.run([tu, ut])  # evaluates both tensors in a single step
sess.close()

# References

[Tensorflow 中eval（）和sess.run()的关系](https://link.zhihu.com/?target=https%3A//blog.csdn.net/zhuiyuanzhongjia/article/details/80463237)

[Tensorflow中 Graph和Session的关系](https://link.zhihu.com/?target=https%3A//blog.csdn.net/yanshuai_tek/article/details/78393559)

[TensorFlow进阶（三）\-\-\-变量的创建、初始化 \- 时间&煮雨~ \- 博客园](https://link.zhihu.com/?target=https%3A//www.cnblogs.com/fwl8888/p/9792706.html)

[Tensorflow学习笔记2：About Session, Graph, Operation and Tensor](https://link.zhihu.com/?target=https%3A//www.cnblogs.com/lienhua34/p/5998853.html)

[TensorFlow学习笔记1：graph、session和op \- Jiax \- 博客园](https://link.zhihu.com/?target=https%3A//www.cnblogs.com/jiaxblog/p/9054051.html)

[TensorFlow学习（三）：Graph和Session \- 谢小小XH \- CSDN博客](https://link.zhihu.com/?target=https%3A//blog.csdn.net/xierhacker/article/details/53860379)